# Scan output_files/output_sheets for CSV Files

This notebook scans the `output_files/output_sheets` directory and lists all `.csv` files it finds.

Future steps: You can expand this notebook to read, process, or plot the data from these files.

In [1]:
# Standard library imports
import os
from pathlib import Path

# Define the directory to scan (relative path)
output_sheets_dir = Path('../output_files/output_sheets')

# List to store found CSV filenames
csv_files = []

# Scan the directory for .csv files
if output_sheets_dir.exists() and output_sheets_dir.is_dir():
    for filename in os.listdir(output_sheets_dir):
        if filename.lower().endswith('.csv'):
            csv_files.append(filename)
else:
    print(f'Directory not found: {output_sheets_dir}')

# Print the found CSV files
print('Found CSV files:')
for fname in csv_files:
    print(f'- {fname}')

# ---
# Future expansion:
# - Read CSV files
# - Process data
# - Plot results

Found CSV files:
- 83034-1 HB015PB01.csv
- 86349-1 HB002PB01.csv
- 86349-1 HB007PB01.csv
- 86349-1 HB009PB01.csv
- 86349-1 HB010PB01.csv
- 86349-1 HB011PB01.csv
- 86349-1 HB013PB01.csv
- 86349-1 HB014PB01.csv
- 86349-1 HB015PB01.csv
- 86349-1 HB017PB01.csv
- 86349-1 HB018PB01.csv
- 86349-1 HB019PB01.csv
- 86349-1 HB021PB01.csv
- 86349-1 HB022PB01.csv
- 86349-1 HB023PB01.csv
- 86349-1 HB025PB01.csv
- 86349-1 HB026PB01.csv
- 86349-1 HB027PB01.csv
- 86349-1 HB029PB01.csv
- 86349-1 HB030PB01.csv
- 86349-1 HB031PB01.csv
- 86349-1 HB033PB01.csv
- 86349-1 HB034PB01.csv
- 86349-1 HB036PB01.csv
- 86349-1 HB037PB01.csv
- 86349-1 HB039PB01.csv
- 86349-1 HB040PB01.csv
- 86349-1 HB041PB01.csv
- 86349-1 HB046PB01.csv
- 86349-1 HB048PB01.csv
- 86349-1 HB050PB01.csv
- 86349-1 HB051PB01.csv
- 86349-1 MB001PB01.csv
- 86349-1 MB001PB02.csv
- 86349-1 MB003PB01.csv
- 86349-1 MB003PB02.csv
- 86349-1 MB005PB01.csv
- 86349-1 MB005PB02.csv
- 86349-1 MB008PB01.csv
- 86349-1 MB008PB02.csv
- 86349-1 MB012PB01.csv

In [2]:
# --- Pastas Model Analysis for Each CSV File ---
import pandas as pd
import pastas as ps
import numpy as np

# Example: set the relative paths to precipitation and evaporation input files
input_prec_path = '../input_files/input_prec/prec_station_249.csv'  # Update filename as needed
input_evap_path = '../input_files/input_evap/evap_station_249.csv'   # Update filename as needed

# Read precipitation and evaporation CSVs as pandas Series, always using first column as date and second as value
def read_timeseries_csv(path):
    df = pd.read_csv(path)
    date_col = df.columns[0]
    value_col = df.columns[1]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    series = df[value_col].dropna()
    return series

# Try reading the files and check if they are pandas Series with DateTimeIndex
try:
    input_prec = read_timeseries_csv(input_prec_path)
    assert isinstance(input_prec, pd.Series) and isinstance(input_prec.index, pd.DatetimeIndex)
    print(f"input_prec loaded: {input_prec.shape}, index type: {type(input_prec.index)}")
except Exception as e:
    print(f"Failed to load input_prec: {e}")

try:
    input_evap = read_timeseries_csv(input_evap_path)
    assert isinstance(input_evap, pd.Series) and isinstance(input_evap.index, pd.DatetimeIndex)
    print(f"input_evap loaded: {input_evap.shape}, index type: {type(input_evap.index)}")
except Exception as e:
    print(f"Failed to load input_evap: {e}")
# waterhoogte_daily_mean = ...  # pd.Series with datetime index (optional extra stressor)


input_prec loaded: (913,), index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
input_evap loaded: (913,), index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [3]:
input_evap.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 913 entries, 2023-01-01 to 2025-07-01
Series name: ET
Non-Null Count  Dtype  
--------------  -----  
913 non-null    float64
dtypes: float64(1)
memory usage: 14.3 KB


In [ ]:
# Ensure csv_files is defined and populated in this cell, in case previous cells were not run
import os
from pathlib import Path

output_sheets_dir = Path('../output_files/output_sheets')
csv_files = []
if output_sheets_dir.exists() and output_sheets_dir.is_dir():
    for filename in os.listdir(output_sheets_dir):
        if filename.lower().endswith('.csv'):
            csv_files.append(filename)
else:
    print(f'Directory not found: {output_sheets_dir}')

# Define model components
recharge_models = {
    "Linear": ps.rch.Linear(),
    "FlexModel": ps.rch.FlexModel(),
    "Berendrecht": ps.rch.Berendrecht()
}
response_functions = {
    "Exponential": ps.Exponential(),
    "Gamma": ps.Gamma(),
    "DoubleExponential": ps.DoubleExponential(),
    "Hantush": ps.Hantush(),
    "FourParam": ps.FourParam(),
}

results = []

# Cap the number of files to process
max_files = 3
csv_files_to_process = csv_files[:max_files]
print(f"Processing up to {max_files} CSV files (found {len(csv_files)})")

# --- Loop over each observation file ---
for csv_file in csv_files_to_process:
    print(f"\n=== Processing file: {csv_file} ===")
    try:
        df = pd.read_csv(output_sheets_dir / csv_file, parse_dates=["Timestamp"])
        df = df.rename(columns={"head": "head_raw"})
        df = df.set_index("Timestamp")
        head_daily_median = df["head_raw"].resample('D').median().dropna()

        # Determine time window for slicing input data
        start_date = head_daily_median.index.min()
        end_date = head_daily_median.index.max()

        # Slice preloaded input data to match this file's time range
        input_prec_slice = input_prec.loc[start_date:end_date].copy()
        input_evap_slice = input_evap.loc[start_date:end_date].copy()
        print(input_prec_slice[0:10])
        print(input_evap_slice[0:10])

        # Optional safety check to ensure alignment
        if not input_prec_slice.index.equals(head_daily_median.index):
            input_prec_slice = input_prec_slice.reindex(head_daily_median.index)
        if not input_evap_slice.index.equals(head_daily_median.index):
            input_evap_slice = input_evap_slice.reindex(head_daily_median.index)

        # Loop over recharge models and response functions
        # 1) Fit all recharge × response variants
        for rch_name, rch_model in recharge_models.items():
            for rfunc_name, rfunc in response_functions.items():
                model_name = f"{rch_name}_{rfunc_name}"
                print(f"  Running model: {model_name}")
                try:
                    ml = ps.Model(head_daily_median, name=model_name)
                    rm = ps.RechargeModel(
                        prec=input_prec_slice,
                        evap=input_evap_slice,
                        recharge=rch_model,
                        rfunc=rfunc,
                        name="rch"
                    )
                    ml.add_stressmodel(rm)
                    ml.add_noisemodel(ps.ArNoiseModel())
                    ml.solve(report=True)
                    stats = ml.stats
                    results.append({
                        "file": csv_file,
                        "model": model_name,
                        "RechargeModel": rch_name,
                        "RechargeRfunc": rfunc_name,
                        "EVP": stats.evp(),
                        "R2": stats.rsq(),
                        "RMSE": stats.rmse(),
                        "AIC": stats.aic(),
                        "BIC": stats.bic()
                    })
                except Exception as e:
                    print(f"    Model {model_name} failed: {e}")
                    results.append({
                        "file": csv_file,
                        "model": model_name,
                        "RechargeModel": rch_name,
                        "RechargeRfunc": rfunc_name,
                        "EVP": None,
                        "R2": None,
                        "RMSE": None,
                        "AIC": None,
                        "BIC": None,
                        "error": str(e)
                    })

        # 2) Now fit the TarsoModel once, using Exponential (the only supported rfunc)
        tarso_name = "Tarso_Exp"
        print(f"\n  Running Tarso model: {tarso_name}")
        try:
            ml2 = ps.Model(head_daily_median, name=tarso_name)
            tm = ps.TarsoModel(
                prec=input_prec_slice,
                evap=input_evap_slice,
                oseries=head_daily_median,  # lets Tarso auto-set dmin/dmax
                rfunc=ps.Exponential(),     # must be Exponential()
                name="tarso"
            )
            ml2.add_stressmodel(tm)
            ml2.add_noisemodel(ps.ArNoiseModel())
            ml2.solve(report=True)
            stats2 = ml2.stats
            results.append({
                "file": csv_file,
                "model": tarso_name,
                "RechargeModel": "Tarso",
                "RechargeRfunc": "Exponential",
                "EVP": stats2.evp(),
                "R2": stats2.rsq(),
                "RMSE": stats2.rmse(),
                "AIC": stats2.aic(),
                "BIC": stats2.bic()
            })
        except Exception as e:
            print(f"    Model {tarso_name} failed: {e}")
            results.append({
                "file": csv_file,
                "model": tarso_name,
                "RechargeModel": "Tarso",
                "RechargeRfunc": "Exponential",
                "EVP": None,
                "R2": None,
                "RMSE": None,
                "AIC": None,
                "BIC": None,
                "error": str(e)
            })

# Combine results into a DataFrame
results_df = pd.DataFrame(results)
print("\nCombined results (first 10 rows):")
print(results_df.head(10))

The Time Series 'Precipitation' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.
The Time Series 'ET' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.


Processing up to 3 CSV files (found 333)

=== Processing file: 83034-1 HB015PB01.csv ===
DATE
2025-04-22    0.0
2025-04-23   -0.1
2025-04-24    4.0
2025-04-25   -0.1
2025-04-26    0.0
2025-04-27    0.0
2025-04-28    0.0
2025-04-29    0.0
2025-04-30    0.0
2025-05-01    0.0
Name: Precipitation, dtype: float64
DATE
2025-04-22    2.352983
2025-04-23    2.971853
2025-04-24    0.742346
2025-04-25    3.512753
2025-04-26    5.607015
2025-04-27    6.555549
2025-04-28    7.115776
2025-04-29    7.130098
2025-04-30    8.060042
2025-05-01    8.647587
Name: ET, dtype: float64
  Running model: Linear_Exponential
    Model Linear_Exponential failed: Need at least 3 dates to infer frequency
  Running Tarso model: Tarso_Exp
    Model Tarso_Exp failed: Need at least 3 dates to infer frequency
  Running model: Linear_Gamma
    Model Linear_Gamma failed: Need at least 3 dates to infer frequency
  Running model: Linear_DoubleExponential
    Model Linear_DoubleExponential failed: Need at least 3 dates to in

The Time Series 'Precipitation' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.
The Time Series 'ET' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.


Fit report Linear_Exponential     Fit Statistics
nfev    48                     EVP         89.80
nobs    326                    R2           0.90
noise   True                   RMSE         0.03
tmin    2024-08-14 00:00:00    AICc     -2674.95
tmax    2025-07-05 00:00:00    BIC      -2656.20
freq    D                      Obj          0.04
warmup  3650 days 00:00:00     ___              
solver  LeastSquares           Interp.        No

Parameters (5 optimized)
                optimal    initial  vary
rch_A          0.208200   0.170185  True
rch_a        437.985129  10.000000  True
rch_f         -1.379591  -1.000000  True
constant_d    -2.744456  -2.965149  True
noise_alpha    4.721268   1.000000  True

Warnings! (2)
Response tmax for 'rch' > than calibration period.
Response tmax for 'rch' > than warmup period.
  Running model: Linear_Gamma


The Time Series 'Precipitation' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.
The Time Series 'ET' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.


...

Fit report Berendrecht_Gamma     Fit Statistics
nfev    80                     EVP         0.00
nobs    122                    R2         -0.27
noise   True                   RMSE        0.21
tmin    2025-03-06 00:00:00    AICc     -937.77
tmax    2025-07-05 00:00:00    BIC      -916.62
freq    D                      Obj         0.02
warmup  3650 days 00:00:00     ___             
solver  LeastSquares           Interp.       No

Parameters (8 optimized)
                 optimal     initial   vary
rch_A           2.080370   13.243921   True
rch_n           0.452540    1.000000   True
rch_a        1048.914042   10.000000   True
rch_fi          0.900000    0.900000  False
rch_fc          1.000000    1.000000  False
rch_sr          0.250000    0.250000  False
rch_de        228.939179  250.000000   True
rch_l          -2.177716    2.000000   True
rch_m           0.500000    0.500000  False
rch_ks         28.206073  100.000000   True
constant_d     -3.828165   -3.861934   True
noise_alpha   

The Time Series 'Precipitation' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.
The Time Series 'ET' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.


Fit report Berendrecht_DoubleExp Fit Statistics
nfev    93                     EVP         0.00
nobs    122                    R2         -0.28
noise   True                   RMSE        0.22
tmin    2025-03-06 00:00:00    AICc     -936.25
tmax    2025-07-05 00:00:00    BIC      -912.62
freq    D                      Obj         0.02
warmup  3650 days 00:00:00     ___             
solver  LeastSquares           Interp.       No

Parameters (9 optimized)
                 optimal     initial   vary
rch_A           0.040326   13.243921   True
rch_alpha       0.010133    0.100000   True
rch_a1          9.206562   10.000000   True
rch_a2          9.845005   10.000000   True
rch_fi          0.900000    0.900000  False
rch_fc          1.000000    1.000000  False
rch_sr          0.250000    0.250000  False
rch_de        286.447540  250.000000   True
rch_l          -1.904628    2.000000   True
rch_m           0.500000    0.500000  False
rch_ks       1956.559798  100.000000   True
constant_d    

The Time Series 'Precipitation' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.
The Time Series 'ET' has nan-values. Pastas will use the fill_nan settings to fill up the nan-values.


Fit report Berendrecht_Hantush   Fit Statistics
nfev    272                    EVP        62.85
nobs    122                    R2          0.51
noise   True                   RMSE        0.13
tmin    2025-03-06 00:00:00    AICc     -938.24
tmax    2025-07-05 00:00:00    BIC      -917.08
freq    D                      Obj         0.02
warmup  3650 days 00:00:00     ___             
solver  LeastSquares           Interp.       No

Parameters (8 optimized)
                 optimal     initial   vary
rch_A        1287.047609   13.243921   True
rch_a         347.173111  100.000000   True
rch_b          25.000000    1.000000   True
rch_fi          0.900000    0.900000  False
rch_fc          1.000000    1.000000  False
rch_sr          0.250000    0.250000  False
rch_de        999.999587  250.000000   True
rch_l          -2.174451    2.000000   True
rch_m           0.500000    0.500000  False
rch_ks         21.278313  100.000000   True
constant_d    -24.783727   -3.861934   True
noise_alpha   

In [5]:
results_df

,file,model,RechargeModel,RechargeRfunc,EVP,R2,RMSE,AIC,BIC,error
0,83034-1 HB015PB01.csv,Linear_Exponential,Linear,Exponential,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
1,83034-1 HB015PB01.csv,Tarso_Exp,Tarso,Exponential,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
2,83034-1 HB015PB01.csv,Linear_Gamma,Linear,Gamma,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
3,83034-1 HB015PB01.csv,Linear_DoubleExponential,Linear,DoubleExponential,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
4,83034-1 HB015PB01.csv,Linear_Hantush,Linear,Hantush,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
5,83034-1 HB015PB01.csv,Linear_FourParam,Linear,FourParam,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
6,83034-1 HB015PB01.csv,FlexModel_Exponential,FlexModel,Exponential,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
7,83034-1 HB015PB01.csv,Tarso_Exp,Tarso,Exponential,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
8,83034-1 HB015PB01.csv,FlexModel_Gamma,FlexModel,Gamma,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
9,83034-1 HB015PB01.csv,FlexModel_DoubleExponential,FlexModel,DoubleExponential,NaN,NaN,NaN,NaN,NaN,Need at least 3 dates to infer frequency
